In [2]:
from pyspark.sql import SparkSession
import random
spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

In [3]:
nation_raw = spark.read.csv("/nation.tbl",sep='|',inferSchema=True)
nation_raw.show()

+---+----------+---+--------------------+----+
|_c0|       _c1|_c2|                 _c3| _c4|
+---+----------+---+--------------------+----+
|  0|   ALGERIA|  0| haggle. carefull...|null|
|  1| ARGENTINA|  1|al foxes promise ...|null|
|  2|    BRAZIL|  1|y alongside of th...|null|
|  3|    CANADA|  1|eas hang ironic, ...|null|
|  4|     EGYPT|  4|y above the caref...|null|
|  5|  ETHIOPIA|  0|ven packages wake...|null|
|  6|    FRANCE|  3|refully final req...|null|
|  7|   GERMANY|  3|l platelets. regu...|null|
|  8|     INDIA|  2|ss excuses cajole...|null|
|  9| INDONESIA|  2| slyly express as...|null|
| 10|      IRAN|  4|efully alongside ...|null|
| 11|      IRAQ|  4|nic deposits boos...|null|
| 12|     JAPAN|  2|ously. final, exp...|null|
| 13|    JORDAN|  4|ic deposits are b...|null|
| 14|     KENYA|  0| pending excuses ...|null|
| 15|   MOROCCO|  0|rns. blithely bol...|null|
| 16|MOZAMBIQUE|  0|s. ironic, unusua...|null|
| 17|      PERU|  1|platelets. blithe...|null|
| 18|     CHI

In [4]:
nation_raw.dtypes

[('_c0', 'int'),
 ('_c1', 'string'),
 ('_c2', 'int'),
 ('_c3', 'string'),
 ('_c4', 'string')]

In [5]:
nation = nation_raw.toDF('NationKey','Name','RegionKey','Comment','extra').drop('extra').cache()
nation.show()

+---------+----------+---------+--------------------+
|NationKey|      Name|RegionKey|             Comment|
+---------+----------+---------+--------------------+
|        0|   ALGERIA|        0| haggle. carefull...|
|        1| ARGENTINA|        1|al foxes promise ...|
|        2|    BRAZIL|        1|y alongside of th...|
|        3|    CANADA|        1|eas hang ironic, ...|
|        4|     EGYPT|        4|y above the caref...|
|        5|  ETHIOPIA|        0|ven packages wake...|
|        6|    FRANCE|        3|refully final req...|
|        7|   GERMANY|        3|l platelets. regu...|
|        8|     INDIA|        2|ss excuses cajole...|
|        9| INDONESIA|        2| slyly express as...|
|       10|      IRAN|        4|efully alongside ...|
|       11|      IRAQ|        4|nic deposits boos...|
|       12|     JAPAN|        2|ously. final, exp...|
|       13|    JORDAN|        4|ic deposits are b...|
|       14|     KENYA|        0| pending excuses ...|
|       15|   MOROCCO|      

In [10]:
def load_dataset_and_set_views(path="/"):
    global supplier, orders, customer, partsupp, nation, part
    
    supplier_raw = spark.read.csv(path+"supplier.tbl",sep='|',inferSchema=True).drop("_c7")
    supplier = supplier_raw.toDF("SuppKey","Name","Address","NationKey","Phone","AcctBal","Comment").cache()
    supplier.createOrReplaceTempView("supplier")
    
    order_raw = spark.read.csv(path+"orders.tbl",sep='|',inferSchema=True).drop("_c9")
    orders = order_raw.toDF("ORDERKEY","CUSTKEY","ORDERSTATUS",
                            "TOTALPRICE","ORDERDATE","ORDERPRIORITY",
                            "CLERK","SHIPPRIORITY","COMMENT").cache()
    orders.createOrReplaceTempView("orders")

    customer_raw = spark.read.csv(path+"customer.tbl",sep='|',inferSchema=True).drop("_c8")
    customer = customer_raw.toDF("CUSTKEY","NAME","ADDRESS","NATIONKEY",
                                "PHONE","ACCTBAL","MKTSEGMENT","COMMENT").cache()
    customer.createOrReplaceTempView("customer")
    
    partsupp_raw = spark.read.csv(path+"partsupp.tbl",sep='|',inferSchema=True).drop("_c5")
    partsupp = partsupp_raw.toDF("PARTKEY","SUPPKEY",
                                 "AVAILQTY","SUPPLYCOST","COMMENT").cache()
    partsupp.createOrReplaceTempView("partsupp")
    
    nation_raw = spark.read.csv(path+"nation.tbl",sep='|',inferSchema=True).drop("_c4")
    nation = nation_raw.toDF("NATIONKEY","NAME","REGIONKEY","COMMENT").cache()
    nation.createOrReplaceTempView("nation")
    
    part_raw = spark.read.csv(path+"part.tbl",sep='|',inferSchema=True).drop("_c9")
    part = part_raw.toDF("PARTKEY","NAME","MFGR","BRAND","TYPE","SIZE",
                        "CONTAINER","RETAILPRICE","COMMENT").cache()
    part.createOrReplaceTempView("part")

In [12]:
def five_highest_totalprice_orders_sql():
    return spark.sql("select ORDERKEY, ORDERDATE, TOTALPRICE from orders ORDER BY TOTALPRICE DESC LIMIT 5")


In [13]:
five_highest_totalprice_orders_sql().show()

+--------+-------------------+----------+
|ORDERKEY|          ORDERDATE|TOTALPRICE|
+--------+-------------------+----------+
|  279812|1994-02-19 00:00:00| 479129.21|
|  370726|1996-09-29 00:00:00|  460099.4|
|   66659|1993-10-15 00:00:00| 458396.42|
|  253639|1998-01-23 00:00:00| 456532.89|
|  502886|1994-04-12 00:00:00| 456423.88|
+--------+-------------------+----------+



In [14]:
def five_highest_totalprice_orders_dtf():
    return orders.select('ORDERKEY','ORDERDATE','TOTALPRICE').orderBy('TOTALPRICE', ascending=False)


In [15]:
five_highest_totalprice_orders_dtf().show(5)

+--------+-------------------+----------+
|ORDERKEY|          ORDERDATE|TOTALPRICE|
+--------+-------------------+----------+
|  279812|1994-02-19 00:00:00| 479129.21|
|  370726|1996-09-29 00:00:00|  460099.4|
|   66659|1993-10-15 00:00:00| 458396.42|
|  253639|1998-01-23 00:00:00| 456532.89|
|  502886|1994-04-12 00:00:00| 456423.88|
+--------+-------------------+----------+
only showing top 5 rows



In [32]:
def cust_most_recent_order_sql(custkey):
    return spark.sql("select c.NAME, o.ORDERDATE, o.TOTALPRICE from orders o INNER JOIN CUSTOMER c ON c.CUSTKEY = o.CUSTKEY where o.CUSTKEY={} ORDER BY ORDERDATE DESC LIMIT 1".format(custkey))


In [33]:
cust_most_recent_order_sql(1).show()


+------------------+-------------------+----------+
|              NAME|          ORDERDATE|TOTALPRICE|
+------------------+-------------------+----------+
|Customer#000000001|1997-03-04 00:00:00| 268835.44|
+------------------+-------------------+----------+



In [34]:
def cust_most_recent_order_dtf(custkey):
    return orders.filter("CUSTKEY = {}".format(custkey)).join(customer, on=['CUSTKEY'], how='inner').select('NAME','ORDERDATE','TOTALPRICE').orderBy('ORDERDATE', ascending=False).limit(1)


In [35]:
cust_most_recent_order_dtf(1).show(1)


+------------------+-------------------+----------+
|              NAME|          ORDERDATE|TOTALPRICE|
+------------------+-------------------+----------+
|Customer#000000001|1997-03-04 00:00:00| 268835.44|
+------------------+-------------------+----------+



In [36]:
def distinct_supplied_parts(nname):
    return spark.sql("""
    SELECT COUNT(DISTINCT P.PARTKEY)
    FROM partsupp p
    INNER JOIN supplier s ON p.SUPPKEY = s.SUPPKEY
    INNER JOIN nation n ON s.NATIONKEY = n.NATIONKEY 
    WHERE n.NAME = '{}'""".format(nname)).first()[0]
    
    #DF
    #return nation.filter("NAME = '{}'".format(nname)).join(supplier, on=['NATIONKEY'], how='inner').join(partsupp, on=['SUPPKEY'], how='inner').select("PARTKEY").distinct().count()

In [37]:
distinct_supplied_parts("CANADA")


2799

In [38]:
def count_suppliers_brand_per_nation(bname):
    a = part.filter("BRAND = '{}'".format(bname)).join(partsupp, on=['PARTKEY'], how='inner').select("SUPPKEY").distinct()
    b = a.join(supplier, on=['SUPPKEY'], how='inner').select('SUPPKEY','NATIONKEY')
    return b.join(nation, on=['NATIONKEY'], how='inner').select('SUPPKEY','NAME').groupBy('NAME').count().orderBy('NAME')

In [39]:
count_suppliers_brand_per_nation("Brand#14").show()

+----------+-----+
|      NAME|count|
+----------+-----+
|   ALGERIA|   34|
| ARGENTINA|   38|
|    BRAZIL|   41|
|    CANADA|   35|
|     CHINA|   51|
|     EGYPT|   39|
|  ETHIOPIA|   32|
|    FRANCE|   35|
|   GERMANY|   49|
|     INDIA|   45|
| INDONESIA|   45|
|      IRAN|   39|
|      IRAQ|   40|
|     JAPAN|   40|
|    JORDAN|   28|
|     KENYA|   35|
|   MOROCCO|   40|
|MOZAMBIQUE|   32|
|      PERU|   37|
|   ROMANIA|   32|
+----------+-----+
only showing top 20 rows



In [40]:
from pyspark.sql.functions import year
def order_number_per_customer_nation(nname):
    a = nation.filter("NAME = '{}'".format(nname)).join(customer, on=['NATIONKEY'], how='inner').select("CUSTKEY")
    b = a.join(orders, on=['CUSTKEY'], how='inner').select(year("ORDERDATE").alias('year')).groupBy('year').count().orderBy('count', ascending=False)
    return b

In [42]:
order_number_per_customer_nation("CANADA").show()

+----+-----+
|year|count|
+----+-----+
|1992|  982|
|1996|  940|
|1995|  932|
|1997|  921|
|1994|  912|
|1993|  900|
|1998|  595|
+----+-----+

